In [31]:
import numpy as np
import pandas as pd 
import sklearn 
import spacy 
import re 
import nltk 
from  nltk.corpus import gutenberg 
from sklearn.feature_extraction.text import CountVectorizer 
nltk.download('gutenberg')

!python -m spacy download en

import warnings 
warnings.filterwarnings('ignore')

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\Sunil\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [32]:
# removing punctuation and numbers from the text

def text_cleaner(text): 
    text = re.sub(r'--',' ', text)
    text = re.sub('[\[].*?[\]]', '', text)
    text = re.sub(r'(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b', '', text)
    text = ' '.join(text.split())
    return text

In [33]:
# loading data
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# dealing with the chapter indicators
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)

alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [34]:
# parsing the text with nlp module

nlp = spacy.load('en_core_web_sm')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)


In [35]:
# groupping the parsed doc into sentences
alice_sents = [[sent, 'Carroll']for sent in alice_doc.sents]
persuasion_sents = [[sent, 'Austen']for sent in persuasion_doc.sents]

# Combining the sentences from two novel to one df
sentences = pd.DataFrame(alice_sents + persuasion_sents, columns= ['text', 'author'])
sentences.head()

,text,author
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [36]:
# removing stopwords and punctuation and lemmatize the token
for i, sentence in enumerate(sentences['text']):
    sentences.loc[i, 'text'] = ' '. join(
    [token.lemma_ for token in sentence if not token.is_punct and not token.is_stop])
    

In [37]:
vectorizer = CountVectorizer(analyzer='word')
X = vectorizer.fit_transform(sentences['text'])
bow_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
sentences = pd.concat([bow_df, sentences[['text', 'author']]], axis = 1)


In [38]:
sentences.head()

,1st,29th,abbreviation,abdication,abide,ability,able,abode,abominable,abominate,...,younker,youth,youthful,zeal,zealand,zealous,zealously,zigzag,text,author
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Alice begin tired sit sister bank have twice p...,Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,consider mind hot day feel sleepy stupid pleas...,Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,remarkable Alice think way hear Rabbit oh dear,Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,oh dear,Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,shall late,Carroll


In [39]:
# Model building with hyperparameter tuning

from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier 
from sklearn.model_selection import train_test_split, GridSearchCV 

Y = sentences['author']
X = np.array(sentences.drop(['text', 'author'], 1))

# splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size= 0.4, random_state= 123)


In [40]:
# Models
lr_params = {"penalty": ["l1", "l2"]}
lr = LogisticRegression()

rfc_params = {"n_estimators": [3, 5, 10, 15],
              "max_depth": [2, 3, 4, 5],
              "min_samples_split": [3, 5, 7, 9]}
rfc = RandomForestClassifier()

gbc_params = {"n_estimators": [3, 5, 10, 15],
              "max_depth": [2, 3, 4, 5],
              "min_samples_split": [3, 5, 7, 9]}
gbc = GradientBoostingClassifier()

clf_lr = GridSearchCV(lr, lr_params, cv=5)
clf_lr.fit(X_train, y_train)

clf_rfc = GridSearchCV(rfc, rfc_params, cv=5)
clf_rfc.fit(X_train, y_train)

clf_gbc = GridSearchCV(gbc, gbc_params, cv=5)
clf_gbc.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(),
             param_grid={'max_depth': [2, 3, 4, 5],
                         'min_samples_split': [3, 5, 7, 9],
                         'n_estimators': [3, 5, 10, 15]})

In [41]:
#print scores

print("----------------------Logistic Regression Scores----------------------")
print('Training set score:', clf_lr.score(X_train, y_train))
print('\nTest set score:', clf_lr.score(X_test, y_test))

print("----------------------Random Forest Scores----------------------")
print('Training set score:', clf_rfc.score(X_train, y_train))
print('\nTest set score:', clf_rfc.score(X_test, y_test))

print("----------------------Gradient Boosting Scores----------------------")
print('Training set score:', clf_gbc.score(X_train, y_train))
print('\nTest set score:', clf_gbc.score(X_test, y_test))

----------------------Logistic Regression Scores----------------------
Training set score: 0.9456145866079849

Test set score: 0.8939679547596607
----------------------Random Forest Scores----------------------
Training set score: 0.7164413706381642

Test set score: 0.7016965127238455
----------------------Gradient Boosting Scores----------------------
Training set score: 0.8314995284501729

Test set score: 0.827521206409048


In [42]:
# using 1-gram and 2-gram as features
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1,2))
X = vectorizer.fit_transform(sentences["text"])
bow_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
sentences = pd.concat([bow_df, sentences[["text", "author"]]], axis=1)
sentences.head()

,1st,29th,29th september,abbreviation,abbreviation living,abdication,abdication neighbour,abide,abide consequence,abide figure,...,zealand australia,zealous,zealous officer,zealous subject,zealously,zealously discharge,zigzag,zigzag go,text,author
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Alice begin tired sit sister bank have twice p...,Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,consider mind hot day feel sleepy stupid pleas...,Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,remarkable Alice think way hear Rabbit oh dear,Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,oh dear,Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,shall late,Carroll


In [43]:
Y = sentences['author']
X = np.array(sentences.drop(['text', 'author'], 1))

# splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size= 0.4, random_state= 123)

# Models
lr_params = {"penalty": ["l1", "l2"]}
lr = LogisticRegression()

rfc_params = {"n_estimators": [3, 5, 10, 15],
              "max_depth": [2, 3, 4, 5],
              "min_samples_split": [3, 5, 7, 9]}
rfc = RandomForestClassifier()

gbc_params = {"n_estimators": [3, 5, 10, 15],
              "max_depth": [2, 3, 4, 5],
              "min_samples_split": [3, 5, 7, 9]}
gbc = GradientBoostingClassifier()

clf_lr = GridSearchCV(lr, lr_params, cv=5)
clf_lr.fit(X_train, y_train)

clf_rfc = GridSearchCV(rfc, rfc_params, cv=5)
clf_rfc.fit(X_train, y_train)

clf_gbc = GridSearchCV(gbc, gbc_params, cv=5)
clf_gbc.fit(X_train, y_train)

#print scores

print("----------------------Logistic Regression Scores----------------------")
print('Training set score:', clf_lr.score(X_train, y_train))
print('\nTest set score:', clf_lr.score(X_test, y_test))

print("----------------------Random Forest Scores----------------------")
print('Training set score:', clf_rfc.score(X_train, y_train))
print('\nTest set score:', clf_rfc.score(X_test, y_test))

print("----------------------Gradient Boosting Scores----------------------")
print('Training set score:', clf_gbc.score(X_train, y_train))
print('\nTest set score:', clf_gbc.score(X_test, y_test))

----------------------Logistic Regression Scores----------------------
Training set score: 0.9625903803835272

Test set score: 0.8906691800188501
----------------------Random Forest Scores----------------------
Training set score: 0.77648538195536

Test set score: 0.7780395852968898
----------------------Gradient Boosting Scores----------------------
Training set score: 0.8314995284501729

Test set score: 0.827521206409048


Here, Random Forest didn't perform good as compared to the model in the checkpoint. GBC did good in the 1-gram and 2-gram. 